In [11]:
import pandas as pd
import folium
import geopandas as gpd
import math
import random

In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('AllPointsAllData.kml', driver='KML')

df_2 = pd.read_excel('AllPointsAllData.xlsx')

In [4]:
ui_sounds = [f'sound{i}.wav' for i in range(1, 7)]

In [5]:
df_3 = pd.concat([df, df_2], axis=1)

In [6]:
df_4 = df_3.drop_duplicates(subset='BEZ')
df_4 = df_4.dropna().reset_index()

In [8]:
points = [df_4.geometry[x].bounds[:2] for x in range(len(df_4.geometry))]
points2 = [[y[1], y[0]] for y in points]

In [48]:
m = folium.Map(location=[48.20, 16.40], tiles='OpenStreetMap', zoom_start=11)


for x in range(len(df_4)):
    info = df_4.iloc[x, 4]
    soundsrc = ui_sounds[random.randint(0, len(ui_sounds) - 1)]
    folium.CircleMarker(
    location=points2[x],
    radius=7,
    popup='</iframe>' \
            f'Diversity - Factor of {info} \
            <br><br> \
            <audio controls src={soundsrc} scrolling="no" \
            frameborder="0" width="340" height="115"></audio>' \
            '</iframe>',
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)
    
m